In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import pymc3 as pm
import seaborn as sns
import theano

In [2]:
from typing import Tuple, Optional
from pathlib import Path

import datetime
import io
import matplotlib.pyplot as plt

import tensorflow as tf
import gpflow

from gpflow.ci_utils import ci_niter, ci_range
from gpflow.config import default_float
from gpflow.utilities import print_summary

import warnings

warnings.filterwarnings('ignore')

In [3]:
# Make tensorboard work inside notebook
output_logdir = "/tmp/tensorboard"

!rm -rf "{output_logdir}"
!mkdir "{output_logdir}"

%load_ext tensorboard
%matplotlib inline


def enumerated_logdir(_logdir_id: int = [0]):
    logdir = Path(output_logdir, str(_logdir_id[0]))
    _logdir_id[0] += 1
    return str(logdir)

In [4]:
# Setup random seeds and default float for gpflow tensors
gpflow.config.set_default_float(np.float64)
np.random.seed(0)
tf.random.set_seed(0)

In [5]:
# Load the diabetes dataset
diabetes = datasets.load_diabetes()
X_org = pd.DataFrame(diabetes['data'], columns = diabetes['feature_names'])
y_org = pd.DataFrame(diabetes['target'], columns = ['y'])
data = X_org
data = data * np.shape(data)[0]
data['y'] = np.log(y_org)
data['sex'][data['sex']>0] = 1
data['sex'][data['sex']<0] = 0
train, test = train_test_split(data, test_size=0.2)

In [6]:
num_features = 5
adam_learning_rate = 0.01
iterations = ci_niter(5)

In [7]:
X = np.array(train)[:,:5]
y = np.expand_dims(np.array(train['y']), axis=1)

In [8]:
k = gpflow.kernels.SquaredExponential()
vgp = gpflow.models.VGP(data=(X, y), likelihood=gpflow.likelihoods.StudentT(), kernel=k, mean_function=None) 

In [9]:
# Stop Adam from optimizing the variational parameters
vgp.q_mu.trainable = False
vgp.q_sqrt.trainable = False

variational_params = [(vgp.q_mu, vgp.q_sqrt)]

adam_opt_for_vgp = tf.optimizers.Adam(adam_learning_rate)
natgrad_opt = gpflow.optimizers.NaturalGradient(gamma=0.1)

In [10]:
iterations = 100
for i in range(iterations):
    adam_opt_for_vgp.minimize(
        lambda: - vgp.log_marginal_likelihood(),
        var_list=vgp.trainable_variables)
    natgrad_opt.minimize(
        lambda: - vgp.log_marginal_likelihood(),
        var_list = variational_params)
    likelihood = vgp.log_likelihood()
    if i%20==0:
        tf.print(f'VGP with NaturalGradient and Adam: iteration {i + 1} likelihood {likelihood:.04f}')

VGP with NaturalGradient and Adam: iteration 1 likelihood -1845.0127
VGP with NaturalGradient and Adam: iteration 21 likelihood -1641.4544
VGP with NaturalGradient and Adam: iteration 41 likelihood -1546.0537
VGP with NaturalGradient and Adam: iteration 61 likelihood -1469.8451
VGP with NaturalGradient and Adam: iteration 81 likelihood -1408.5408


In [11]:
# simulate from GP
param = [vgp.trainable_variables[0].numpy(),vgp.trainable_variables[1].numpy()]

def cov_kernel(x1,x2,variance=1.0,lengthscale=1.0): 
    """
    Squared-Exponential covariance kernel
    """ 
    k12 = variance*np.exp(-.5*np.sum((x1 - x2)**2)/lengthscale**2)
    return k12

def make_K(x,variance=1.0,lengthscale=1.0):
    """
    Make covariance matrix from covariance kernel
    """
    # for a data array of length x, make a covariance matrix x*x:
    K = np.zeros((np.shape(x)[0],np.shape(x)[0]))
 
    for i in range(0,len(x)):
        for j in range(0,len(x)):
            # calculate value of K for each separation:
            K[i,j] = cov_kernel(x[i,:],x[j,:],variance,lengthscale)
    return K

np.random.seed(0)
# draw samples from a co-variate Gaussian
K = make_K(X,vgp.kernel.variance.numpy(),vgp.kernel.lengthscale.numpy())
f_sim =  np.random.multivariate_normal(np.zeros(np.shape(X)[0]),K)

# draw noise
noise_sim = np.random.standard_t(param[0],np.shape(X)[0])
y_sim = np.expand_dims(f_sim + noise_sim, axis=1)

In [12]:
# perform VGP on simulated data
k2 = gpflow.kernels.SquaredExponential(vgp.kernel.variance.numpy(), vgp.kernel.lengthscale.numpy())
vgp2 = gpflow.models.VGP(data=(X, y_sim), likelihood=gpflow.likelihoods.StudentT(), kernel=k2, mean_function=None) 

variational_params2 = [(vgp2.q_mu, vgp2.q_sqrt)]

natgrad_opt = gpflow.optimizers.NaturalGradient(gamma=1.0)

for i in range(iterations):
    natgrad_opt.minimize(
        lambda: - vgp2.log_marginal_likelihood(),
        var_list = variational_params2)
    likelihood = vgp2.log_likelihood()
    if i%20==0:
        tf.print(f'VGP with NaturalGradient and Adam: iteration {i + 1} likelihood {likelihood:.04f}')

VGP with NaturalGradient and Adam: iteration 1 likelihood -835.4203
VGP with NaturalGradient and Adam: iteration 21 likelihood -822.2406
VGP with NaturalGradient and Adam: iteration 41 likelihood -822.2406
VGP with NaturalGradient and Adam: iteration 61 likelihood -822.2406
VGP with NaturalGradient and Adam: iteration 81 likelihood -822.2406


In [13]:
f_post = np.squeeze(vgp2.predict_f_samples(predict_at=X).numpy(),axis=0)

In [14]:
import pickle

gp_dict = {'f_prior': f_sim, 'f_post': f_post}

with open('gp_dict','wb') as handle:
    pickle.dump(gp_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'pickle' is not defined